In [1]:
import numpy as np
import pandas as pd
import os,sys
import cv2 as cv
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Input,Dropout
from tensorflow.keras.models import Model

In [8]:
from glob import glob
IMG_SHAPE=[96,96,3]
num_classes=120
train_data_path='dog-breed-identification/train'
#train_label_path='dog-breed-identification\train'
test_data_path='dog-breed-identification/test'
label_path='dog-breed-identification\labels.csv'

In [9]:
preTrainedModel= tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                     include_top=False,
                     weights='imagenet',
                     classes=num_classes)

9412608/9406464 [==============================] - 46s 5us/step


In [10]:
def classifier(preTrainedModel):
    preTrainedModel.trainable=False
    i=Input(shape=preTrainedModel.output_shape[1:],name='top_model_input')
    x=Flatten(name='top_model_flatten')(i)
    x=Dense(256,activation='relu')(x)
    x=Dropout(0.4,name='top_model_dropout')(x)
    x=Dense(num_classes,activation='sigmoid',name='top_model_output')(x)
    top_model=Model(inputs=i,outputs=x)
    print(top_model.summary())
    model=Model(inputs=preTrainedModel.inputs,outputs=top_model(preTrainedModel.outputs),name='classifier')
    return model

In [11]:
classifier_model=classifier(preTrainedModel)
classifier_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
top_model_input (InputLayer) [(None, 3, 3, 1280)]      0         
_________________________________________________________________
top_model_flatten (Flatten)  (None, 11520)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               2949376   
_________________________________________________________________
top_model_dropout (Dropout)  (None, 256)               0         
_________________________________________________________________
top_model_output (Dense)     (None, 120)               30840     
Total params: 2,980,216
Trainable params: 2,980,216
Non-trainable params: 0
_________________________________________________________________
None
Model: "classifier"
____________________________________________________________________________________________

Total params: 5,238,200
Trainable params: 2,980,216
Non-trainable params: 2,257,984
__________________________________________________________________________________________________


In [12]:
classifier_model.compile(loss='sparse_categorical_crossentropy',
                        optimizer=tf.keras.optimizers.SGD(learning_rate=0.0001,momentum=0.9),
                        metrics=['accuracy'])

In [13]:
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                               shear_range=0.2,
                                               zoom_range=0.2,
                                               horizontal_flip=True)

In [14]:
labels=pd.read_csv(label_path)
labels=labels.set_index('id')
class_names=labels['breed'].unique()
num_classes=len(class_names)
index_to_classes={i:c for i,c in enumerate(class_names)}
classes_to_index={c:i for i,c in index_to_classes.items()}
print("{} classes".format(num_classes))

120 classes


In [15]:
all_ys=[]
all_xs=[]
breeds=labels['breed']
for img_path in os.listdir(train_data_path):
    
    class_name=img_path.split(".")[0]
    y=classes_to_index[breeds[class_name]]
    img=tf.keras.preprocessing.image.load_img(os.path.join(train_data_path,img_path),target_size=IMG_SHAPE[0:2])
    x=tf.keras.preprocessing.image.img_to_array(img)
    all_ys.append(y)
    all_xs.append(x)

In [16]:
X_train=np.array(all_xs,dtype=np.uint16)
y_train=np.array(all_ys)
print("X_train Shape:",X_train.shape)
print("Y_train Shape:",y_train.shape)

X_train Shape: (10222, 96, 96, 3)
Y_train Shape: (10222,)


In [17]:
train_generator=train_datagen.flow(X_train,y_train,batch_size=64)

In [ ]:
classifier_model.fit(train_generator,validation_data=(X_train[:100]/255,y_train[:100]),steps_per_epoch=10222//64,verbose=2,epochs=1000)

  ...
    to  
  ['...']
Train for 159 steps, validate on 100 samples
Epoch 1/1000
159/159 - 608s - loss: 4.9764 - accuracy: 0.0087 - val_loss: 4.8058 - val_accuracy: 0.0100
Epoch 2/1000
159/159 - 563s - loss: 4.8581 - accuracy: 0.0113 - val_loss: 4.7312 - val_accuracy: 0.0100
Epoch 3/1000
159/159 - 519s - loss: 4.7818 - accuracy: 0.0144 - val_loss: 4.6799 - val_accuracy: 0.0300
Epoch 4/1000
159/159 - 378s - loss: 4.7432 - accuracy: 0.0156 - val_loss: 4.6431 - val_accuracy: 0.0400
Epoch 5/1000
159/159 - 382s - loss: 4.7069 - accuracy: 0.0218 - val_loss: 4.6073 - val_accuracy: 0.0400
Epoch 6/1000
159/159 - 373s - loss: 4.6821 - accuracy: 0.0254 - val_loss: 4.5755 - val_accuracy: 0.0400
Epoch 7/1000
159/159 - 366s - loss: 4.6526 - accuracy: 0.0292 - val_loss: 4.5424 - val_accuracy: 0.0400
Epoch 8/1000
159/159 - 366s - loss: 4.6313 - accuracy: 0.0322 - val_loss: 4.5138 - val_accuracy: 0.0500
Epoch 9/1000
159/159 - 370s - loss: 4.5956 - accuracy: 0.0325 - val_loss: 4.4875 - val_accuracy: 0

In [ ]:
classifier_model.save('classifier_model.h5')

In [ ]:
if not os.path.exists('model'):
    os.mkdir('model')
tf.saved_model.save(classifier_model,'model')